In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/"My Drive"/"Master Thesis"/"Model"

/content/drive/My Drive/Master Thesis/Model


Importing dependencies numpy and keras

In [3]:
import numpy
from module import Environment, Vehicle, Simulation
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, SimpleRNN
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


生成時間序列資料 load traffic data

In [4]:
sim = Simulation()
X, y, env_list = sim.data_generator(base_dir="../Data", input_var=None)

# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.33, random_state=42
# )

macbeth.txt, 02-section, env_x.shape: (3328, 5, 36)
env_y.shape: (3328, 5, 2)
01-section, env_x.shape: (5081, 5, 36)
env_y.shape: (5081, 5, 2)
x.shape: (8409, 5, 36)
y.shape: (8409, 5, 2)


In [5]:
X[0, 0]

array([9.000000000000002, -0.03000000000001002, 'motor',
       5.1000000000000005, 0.1419999999999999, 5.101976479757625,
       10.199999999999996, 0.23999999999999355, 'motor',
       11.809999999999999, 1.1239999999999997, 11.863366975694547,
       11.100000000000003, -0.1049999999999951, 'motor', 3.54, -1.185,
       3.733071791433966, 9.300000000000002, 0.10500000000000842, 'car',
       1.97, 3.425, 3.951142239909872, 8.700000000000001,
       -0.05999999999999339, nan, nan, nan, nan, nan, nan,
       3.7910000000000004, 0.0, 2.599, 5.4190000000000005], dtype=object)

Some parameters

In [0]:
BATCH_START = 0
# TIME_STEPS = X_train.shape[1]
TIME_STEPS = X.shape[1]
BATCH_SIZE = 20
# INPUT_SIZE = X_train.shape[2]
INPUT_SIZE = X.shape[2]
# OUTPUT_SIZE = y_train.shape[2]
OUTPUT_SIZE = y.shape[2]
CELL_SIZE = 20      # hidden layer unit數
LR = 6E-3
EPOCHS = 50

Constructing the LSTM model

In [0]:
model = Sequential()
model.add(
    LSTM(
        300,
        input_shape=(TIME_STEPS, INPUT_SIZE),
        return_sequences=True
    )
)
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(
    Dense(
        OUTPUT_SIZE,
        activation=None
    )
) 

In [11]:
from keras.utils import plot_model

plot_model(model, to_file='./model.png')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 300)            404400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 300)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 602       
Total params: 1,126,202
Trainable params: 1,126,202
Non-trainable params: 0
_________________________________________________________________


Loss function: x, y座標的 loss

In [0]:
from keras import backend as K

def loss_function(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))    # RMSE

In [0]:
model.compile(
    loss=loss_function, 
    optimizer=Adam(LR)
)

Training log file

In [0]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(
    filename="./model_test.log",
    separator=',',
    append=True
)

Training

In [0]:
history = model.fit(
    X=X,
    y=y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[csv_logger],
    validation_split=0.33
)